In [1]:
!pip install pycocotools matplotlib opencv-python torch torchvision

In [14]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
import os
from PIL import Image
from pycocotools.coco import COCO
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim

In [10]:
# === CONFIG ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BASE_PATH = "/Users/hardikm-visiobyte/Desktop/Oral_Cancer_Ampire/dataset"
BATCH_SIZE = 16
NUM_EPOCHS = 5
NUM_CLASSES = 3  # Abnormal, Normal, Null

In [11]:
# === Custom Dataset ===
class OralCancerCOCODataset(Dataset):
    def __init__(self, folder):
        self.img_dir = os.path.join(BASE_PATH, folder)
        self.ann_path = os.path.join(self.img_dir, '_annotations.coco.json')
        self.coco = COCO(self.ann_path)
        self.image_ids = list(self.coco.getImgIds())

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        # Loop until a valid label is found (excluding category_id = 0)
        while True:
            img_id = self.image_ids[idx]
            img_info = self.coco.loadImgs(img_id)[0]
            img_path = os.path.join(self.img_dir, img_info['file_name'])

            ann_ids = self.coco.getAnnIds(imgIds=img_id)
            anns = self.coco.loadAnns(ann_ids)

            if len(anns) == 0:
                idx = (idx + 1) % len(self.image_ids)
                continue

            label = anns[0]['category_id']
            if label == 0:
                idx = (idx + 1) % len(self.image_ids)
                continue  # skip invalid category

            image = Image.open(img_path).convert('RGB')
            image = self.transform(image)

            label = label - 1  # Make Abnormal=0, Normal=1, Null=2
            return image, label


In [12]:
# === Loaders ===
train_loader = DataLoader(OralCancerCOCODataset('train'), batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(OralCancerCOCODataset('valid'), batch_size=BATCH_SIZE)


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [15]:
# === Model ===
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/hardikm-visiobyte/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%


In [16]:
# === Loss & Optimizer ===
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [17]:
# === Training Loop ===
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch [{epoch + 1}/{NUM_EPOCHS}] Loss: {total_loss:.4f} | Accuracy: {acc:.2f}%")

Epoch [1/5] Loss: 16.2039 | Accuracy: 28.42%
Epoch [2/5] Loss: 3.4937 | Accuracy: 96.17%
Epoch [3/5] Loss: 1.1021 | Accuracy: 98.91%
Epoch [4/5] Loss: 0.4175 | Accuracy: 100.00%
Epoch [5/5] Loss: 0.3242 | Accuracy: 100.00%


In [18]:

# === Save the model ===
torch.save(model.state_dict(), "oral_cancer_classifier.pth")
print("✅ Model saved as 'oral_cancer_classifier.pth'")

✅ Model saved as 'oral_cancer_classifier.pth'
